In [1]:
import datasets
import numpy as np
from tqdm.notebook import tqdm
import csv
import pandas as pd

from substitutions import tenk_word_pairs as word_pairs

In [2]:
ds_path = '../data/17e7_tokens_perturbed.hf/'
num_proc = 16
n_per_sub = 1000
seed = 0

In [3]:
ds = datasets.load_from_disk(ds_path)

In [4]:
ds

Dataset({
    features: ['text', 'meta', 'hash', 'is_original', 'substitutions', 'order'],
    num_rows: 989378
})

In [5]:
swap_arr = np.array(ds["substitutions"])
print(swap_arr.shape)

(989378, 45)


In [9]:
# This random state allows the perturbations to be reproducible
rs = np.random.RandomState(seed=0)

#take the sequences to perturb
do_sub = []
examples = []
for i, (w1, w2) in tqdm(enumerate(word_pairs), total=len(word_pairs)):
    # create indices
    idx = np.arange(len(swap_arr))
    has_sub = idx[swap_arr[:, i] == 1]
    rs.shuffle(has_sub)
    
    all_indexes = has_sub[:n_per_sub]
    labels = rs.randint(0, 2, size=n_per_sub).astype(bool)
    
    # just for checksum
    do_sub.append(all_indexes[labels])
    subset_ds = ds.select(all_indexes)
    
    # assert that all examples received the appropriate substitution
    # assert(all([ str(i) in j for j in ds.select(do_sub[-1])['order']]))
    # assert(all([ f':{i}:' not in '{j}:' for j in ds.select(no_sub)['order']]))
    
    for ex_idx, j, label in zip(all_indexes, subset_ds, labels):
        current_word = w2 if label else w1
        examples.append((ex_idx, j['text'], j['text'].index(f' {current_word} '), w1, w2, label))

  0%|          | 0/45 [00:00<?, ?it/s]

In [10]:
np.sum([ i.sum() for i in do_sub ])

11058546234

In [ ]:
df = pd.DataFrame(examples)
df.columns = ['example_index', 'text', 'sub_index', 'original', 'synonym', 'substituted?']
df.to_csv('./propagation_inputs.csv', index=False)

In [ ]:
df.head(1)